In [5]:
import sys
import os
import numpy as np
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle
import metpy.calc as mpc
from metpy.units import units
# for animations
from PIL import Image

In [6]:
warnings.filterwarnings('ignore')

In [7]:
# aggregation indices
os.chdir('/w2-data2/willytsai/python_module')
import conorgidx_revise as agi
from SCAI_calc4obj import labeled_obj, SCAI_calc
from COP_calc4obj import COP
from MICA_calc4obj import MICA

In [8]:
def read_data(data_dir,t_start=0):
    os.chdir(data_dir)
    print(data_dir)
    file_name = glob('*nc')
    data_zon = xr.open_dataset('cm1out_zonmean.nc')
    data_2d = xr.open_dataset('cm1out_2Dvars.nc')
    th = data_zon.th[t_start:,:,:,:]
    qv = data_zon.qv[t_start:,:,:,:]
    u = data_zon.uinterp[t_start:,:,:,:]
    v = data_zon.vinterp[t_start:,:,:,:]
    w = data_zon.winterp[t_start:,:,:,:]
    qc = data_zon.qc[t_start:,:,:,:]
    qi = data_zon.qi[t_start:,:,:,:]
    prs = data_zon.prs[t_start:,:,:,:]
    # 2d data
    prate = data_2d.prate[t_start:,:,:]
    pwat = data_2d.pwat[t_start:,:,:]
    cwp = data_2d.cwp[t_start:,:,:]
    cape = data_2d.cape[t_start:,:,:]
    cin = data_2d.cin[t_start:,:,:]
    
    # temperature conversion
    T = th/((1000*100/prs)**(287.15/1004))-273.15 # [degC]
    # density 
    ro = prs/(287.15*(T+273.15))

    x_dim = data_2d.lon
    y_dim = data_2d.lat
    div = np.gradient(v,y_dim*1000,axis=2) # du/dx+dv/dy 
    vor = -np.gradient(u,y_dim*1000,axis=2) # -du/dy+dv/dx
    #relative humidty calculation, Buck (1996)
    es_liq = 0.61121*np.exp((18.678-T/234.5)*(T/(257.14+T)))*10 #[hpa]
    es_ice = 0.61115*np.exp((23.036-T/333.7)*(T/(279.82+T)))*10

    qs_liq = 0.622*es_liq/(prs/100-es_liq)
    qs_ice = 0.622*es_ice/(prs/100-es_ice)

    rh = qv/qs_liq
    rh_ice = qv/qs_ice

    rh = rh.values.flatten()
    rh_ice = rh_ice.values.flatten()
    T_test = T.values.flatten()

    rh[T_test<0] = rh_ice[T_test<0]
    rh = rh.reshape((T.shape[0],T.shape[1],T.shape[2],T.shape[3]))
    rh = xr.DataArray(rh,coords=[T.time,T.lev,T.lat,T.lon],dims=['time','lev','lat','lon'])
    
    return (th,T,qv,qc,qi,rh,prate*3600*24,pwat*1000,cwp,u,v,w,div,vor,cape,cin,prs,ro)

In [16]:
# get weather or weather2 
#server = '/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/'
server = '/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne'
os.chdir(server)

exp_name = ['CM1_LD2km_RAD4K_MPICTRL_216x216_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U15H2km_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U12H2km_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U7H2km_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U5H2km_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_narrow',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_wide']
print(exp_name)
print('Number of Exp: ', len(exp_name))

['CM1_LD2km_RAD4K_MPICTRL_216x216_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U15H2km_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U12H2km_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U7H2km_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U5H2km_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_narrow', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_wide']
Number of Exp:  8


In [17]:
(th1,T1,qv1,qc1,qi1,rh1,prate1,pwat1,cwp1,u1,v1,w1,div1,vor1,cape1,cin1,prs1,ro1) = read_data(server+'/'+exp_name[0],t_start=-721)
(th2,T2,qv2,qc2,qi2,rh2,prate2,pwat2,cwp2,u2,v2,w2,div2,vor2,cape2,cin2,prs2,ro2) = read_data(server+'/'+exp_name[1])
(th3,T3,qv3,qc3,qi3,rh3,prate3,pwat3,cwp3,u3,v3,w3,div3,vor3,cape3,cin3,prs3,ro3) = read_data(server+'/'+exp_name[2])
(th4,T4,qv4,qc4,qi4,rh4,prate4,pwat4,cwp4,u4,v4,w4,div4,vor4,cape4,cin4,prs4,ro4) = read_data(server+'/'+exp_name[3])
(th5,T5,qv5,qc5,qi5,rh5,prate5,pwat5,cwp5,u5,v5,w5,div5,vor5,cape5,cin5,prs5,ro5) = read_data(server+'/'+exp_name[4])
(th6,T6,qv6,qc6,qi6,rh6,prate6,pwat6,cwp6,u6,v6,w6,div6,vor6,cape6,cin6,prs6,ro6) = read_data(server+'/'+exp_name[5])
(th7,T7,qv7,qc7,qi7,rh7,prate7,pwat7,cwp7,u7,v7,w7,div7,vor7,cape7,cin7,prs7,ro7) = read_data(server+'/'+exp_name[6])
(th8,T8,qv8,qc8,qi8,rh8,prate8,pwat8,cwp8,u8,v8,w8,div8,vor8,cape8,cin8,prs8,ro8) = read_data(server+'/'+exp_name[7])

# (th5,T5,qv5,qc5,qi5,prate5,pwat5,cwp5,u5,v5,w5,div5,vor5,cape5,cin5) = read_data(server+'/'+exp_name[4])

/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_LD2km_RAD4K_MPICTRL_216x216_eqtset2
/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_LD2km_RAD4K_OPENMPshear_216x216_U15H2km_eqtset2
/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_LD2km_RAD4K_OPENMPshear_216x216_U12H2km_eqtset2
/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_eqtset2
/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_LD2km_RAD4K_OPENMPshear_216x216_U7H2km_eqtset2
/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_LD2km_RAD4K_OPENMPshear_216x216_U5H2km_eqtset2
/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_narrow
/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_wide


In [20]:
# aggregation indices calculation
def AGG_index_time_prec(prec_data,t):

    prec =  prec_data[t,:,:].squeeze() # precip 2D array
    
    Iorg_time = np.zeros(2)*np.nan # 1. Iorg
    SCAI_time = np.copy(Iorg_time) # 2. SCAI
    COP_time = np.copy(Iorg_time) # 3. COP
    MICA_time = np.copy(Iorg_time) # 4. MICA
        
    pr_s = prec[65:151,:] # only in the shear region
    pr_ns = np.concatenate((prec[-43:,:],prec[:43,:]))
    cldmask_s = np.zeros(pr_s.shape)
    cldmask_ns = np.copy(cldmask_s)
    cldmask_s[pr_s> 5] = 1 # cloudmaks, prec>5mm/d, cold cloud
    cldmask_ns[pr_ns> 5] = 1 # cloudmaks, prec>5mm/d, cold cloud

#    Iorg_time[0] = agi.iorg(cldmask_s)
#    Iorg_time[1] = agi.iorg(cldmask_ns)

    labeled_array, num_feature = labeled_obj(cldmask_s,1,1)
    SCAI_time[0] = SCAI_calc(labeled_array, num_feature, dx=1 ,L=216)

    labeled_array, num_feature = labeled_obj(cldmask_ns,1,1)
    SCAI_time[1] = SCAI_calc(labeled_array, num_feature, dx=1 ,L=216)

    COP_time[0] = COP(cldmask_s)
    COP_time[1] = COP(cldmask_ns)
            
    return Iorg_time, SCAI_time, COP_time

In [55]:
# aggregation indices calculation
def AGG_index_time_prec3(prec_data,t):

    prec =  prec_data[t,:,:].squeeze() # precip 2D array
    
    Iorg_time = np.zeros(2)*np.nan # 1. Iorg
    SCAI_time = np.copy(Iorg_time) # 2. SCAI
    COP_time = np.copy(Iorg_time) # 3. COP
    MICA_time = np.copy(Iorg_time) # 4. MICA
        
    pr_s = prec[43:173,:] # only in the shear region
    pr_ns = np.concatenate((prec[-21:,:],prec[:21,:]))
    cldmask_s = np.zeros(pr_s.shape)
    cldmask_ns = np.zeros(pr_ns.shape)
    cldmask_s[pr_s> 5] = 1 # cloudmaks, prec>5mm/d, cold cloud
    cldmask_ns[pr_ns> 5] = 1 # cloudmaks, prec>5mm/d, cold cloud

#    Iorg_time[0] = agi.iorg(cldmask_s)
#    Iorg_time[1] = agi.iorg(cldmask_ns)

    labeled_array, num_feature = labeled_obj(cldmask_s,1,1)
    SCAI_time[0] = SCAI_calc(labeled_array, num_feature, dx=1 ,L=216*2)

    labeled_array, num_feature = labeled_obj(cldmask_ns,1,1)
    SCAI_time[1] = SCAI_calc(labeled_array, num_feature, dx=1 ,L=216/2)

    COP_time[0] = COP(cldmask_s)
    COP_time[1] = COP(cldmask_ns)
            
    return Iorg_time, SCAI_time, COP_time

In [21]:
from functools import partial
from multiprocessing import Pool

In [22]:
x_dim = pwat1.lon
y_dim = pwat1.lat
z_dim = T1.lev
t_dim = np.arange(len(T1.time))/3

In [23]:
exp_outname=['CTRL','U15','U12','U10','U7','U5','U10_narrow','U10_wide']

In [57]:
%%time
Iorg_pr_time = np.zeros((2,len(t_dim)))
SCAI_pr_time = np.zeros((2,len(t_dim)))
COP_pr_time = np.zeros((2,len(t_dim)))

for n,prec in enumerate([prate8]):

    if __name__ == "__main__":
        pool = Pool()
        func = partial(AGG_index_time_prec3, prec)
        time_result = pool.map(func, range(721), chunksize=20)
        pool.close()
        pool.join()
        
    for t in range(len(t_dim)):
        #Iorg_pr_time[n,:,t] = time_result[t][0]
        SCAI_pr_time[:,t] = time_result[t][1]
        COP_pr_time[:,t] = time_result[t][2]

    np.save('/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/SCAI_com_shear_'+exp_outname[7]
            +'.npy',SCAI_pr_time)
    np.save('/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/COP_com_shear_'+exp_outname[7]
            +'.npy',COP_pr_time)

CPU times: user 6.55 s, sys: 5.07 s, total: 11.6 s
Wall time: 42.4 s


In [11]:
np.save('/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/Iorg_com_UNI.npy',Iorg_pr_time)
np.save('/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/COP_com_UNI.npy',COP_pr_time)
np.save('/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/SCAI_com_UNI.npy',SCAI_pr_time)